In [1]:
import pandas as pd

In [59]:
df = pd.read_csv('season1.csv',index_col=0)

def clean_separate_df(df):
    all_players = df[df.year==2011][['player','pos','adp','ppr_pts']].sort_values('adp').reset_index(drop=True)
    qb = all_players[all_players.pos=='QB']
    rb = all_players[all_players.pos=='RB']
    wr = all_players[all_players.pos=='WR']
    te = all_players[all_players.pos=='TE']
    return all_players, qb, rb, wr, te
    
all_players, qb, rb, wr, te = clean_separate_df(df)

In [245]:
rb_heavy_strat = ['RB','RB','RB','WR','QB','WR','RB','TE']
wr_heavy_strat = ['WR','WR','RB','RB','QB','WR','RB','TE']
early_qb_strat = ['RB','QB','WR','RB','WR','WR','TE','RB']
early_te_strat = ['TE','RB','WR','RB','QB','WR','WR','RB']
value_strat = ['VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL']

# Turn this into a function
strats = [('rb_heavy',rb_heavy_strat.copy()),('value_strat',value_strat.copy()),('value_strat',value_strat.copy())
         ,('value_strat',value_strat.copy()),('value_strat',value_strat.copy()),('value_strat',value_strat.copy())
         ,('value_strat',value_strat.copy()),('value_strat',value_strat.copy()),('value_strat',value_strat.copy())
         ,('value_strat',value_strat.copy()),('value_strat',value_strat.copy()),('value_strat',value_strat.copy())]

class Cheatsheet:
    def __init__(self, all_players, qb, rb, wr, te):
        self.all_players = all_players
        self.qb = qb
        self.rb = rb
        self.wr = wr
        self.te = te
      
    def draft_qb(self,owner):
        selection = self.qb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.qb = self.qb[1:]
        owner.qb_count+=1
        return selection
        
    def draft_rb(self,owner):
        selection = self.rb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.rb = self.rb[1:]
        owner.rb_count+=1
        return selection
    
    def draft_wr(self,owner):
        selection = self.wr.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.wr = self.wr[1:]
        owner.wr_count+=1
        return selection
    
    def draft_te(self,owner):
        selection = self.te.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.te = self.te[1:]
        owner.te_count+=1
        return selection
    
    def draft_not_qb(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'QB':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_te(owner)
    
    def draft_not_te(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'TE':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_qb(owner)
        
    def draft_not_wr(self,owner):
        pass
    
    def draft_value(self,owner): 
        selection = self.all_players.iloc[0,:]
        if self.all_players.iloc[0,:].pos == 'QB':
            self.qb = self.qb.drop(index = selection.name)
            owner.qb_count+=1
        elif self.all_players.iloc[0,:].pos == 'RB':
            self.rb = self.rb.drop(index = selection.name)
            owner.rb_count+=1
        elif self.all_players.iloc[0,:].pos == 'WR':
            self.wr = self.wr.drop(index = selection.name)
            owner.wr_count+=1
        else:
            self.te = self.te.drop(index = selection.name)
            owner.te_count+=1
        self.all_players = self.all_players.iloc[1:]
        return selection
    

class Team_owner:
    def __init__(self, strat, draft_pos):
        self.team = []
        self.strat_name = strat[0]      
        self.strat_order = strat[1]
        self.draft_pos = draft_pos
        self.qb_count = 0
        self.te_count = 0
        self.wr_count = 0
        self.rb_count = 0
        
        
    def draft_player(self,cheatsheet):
        if len(self.team) == 7 and self.te_count == 0:
            self.strat_order.pop(0)
            return cheatsheet.draft_te(self)
        
        elif self.qb_count == 2 and self.strat_order[0]=='VAL':
            self.strat_order.pop(0)
            return cheatsheet.draft_not_qb(self)
        
        elif self.te_count == 2 and self.strat_order[0]=='VAL':
            self.strat_order.pop(0)
            return cheatsheet.draft_not_te(self)
        
        elif len(self.team) == 4 and self.qb_count == 0:
            self.strat_order.pop(0)
            return cheatsheet.draft_qb(self)
        
        else:
            pos = self.strat_order.pop(0)
            if pos == 'QB':
                self.qb_count+=1
                return cheatsheet.draft_qb(self)
            elif pos == 'RB':
                return cheatsheet.draft_rb(self)
            elif pos == 'WR':
                return cheatsheet.draft_wr(self)
            elif pos == 'TE':
                return cheatsheet.draft_te(self)
            else:
                return cheatsheet.draft_value(self)


    
            
    

In [233]:
# cheat = Cheatsheet(all_players, qb, rb, wr, te)
# o = Team_owner(strats.copy()[1],1)

In [246]:
cheat = Cheatsheet(all_players, qb, rb, wr, te) # Create new cheat sheet

owners = [] #Create 12 team owners with 'value_strat' drafting strategy
for i, s in zip(range(0,12),strats.copy()):
    owners.append(Team_owner(s,i+1))

In [247]:
for i in range(0,8): #Draft 8 rounds for each team owner
    owners.reverse()
    for o in owners:
        o.team.append(o.draft_player(cheat))

In [252]:
owners[1].qb_count

1

In [253]:
all_teams = []

for o in owners:
    df = pd.concat(o.team,axis=1).T
    df['draft_pos'] = o.draft_pos
    df['draft_strat'] = o.strat_name
    all_teams.append(df)

df = pd.concat(all_teams)

In [256]:
df[df.draft_pos==1].sort_values(['pos','ppr_pts'],ascending=False)

,player,pos,adp,ppr_pts,draft_pos,draft_strat
37,Dwayne Bowe,WR,38.0,228.1,1,rb_heavy
54,Mario Manningham,WR,55.0,115.3,1,rb_heavy
82,Kellen Winslow,TE,85.0,165.3,1,rb_heavy
35,Ahmad Bradshaw,RB,36.0,192.6,1,rb_heavy
12,Rashard Mendenhall,RB,13.0,178.2,1,rb_heavy
11,Darren McFadden,RB,12.0,123.8,1,rb_heavy
80,Daniel Thomas,RB,82.0,81.3,1,rb_heavy
97,Eli Manning,QB,103.0,280.8,1,rb_heavy


In [241]:
df.groupby('draft_pos')['ppr_pts'].sum()

draft_pos
1     1365.4
2     1351.5
3     1538.8
4     1826.2
5     1653.0
6     1442.2
7     1522.4
8     1141.4
9     1662.3
10    1816.8
11    1693.3
12    1272.9
Name: ppr_pts, dtype: object